In [ ]:
import pandas as pd

: 

In [ ]:
df_real=pd.read_csv('fake.csv')
df_fake=pd.read_csv('real.csv')

In [ ]:
df_real['label']=0
df_fake['label']=1
df=pd.concat([df_real,df_fake],ignore_index=True)


In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

,index,tweet,label
0,0,"the tongue of his condition, now i saw things ...",0
1,1,"by god the great, after i saw the derby of wyd...",0
2,2,"believe in god, this zamalek fans are as good ...",0
3,3,the coach appeared in the media and told the t...,0
4,4,asno smito\nwhoever knows her tells me a massa...,0


In [ ]:
df.shape

(41868, 3)

In [ ]:
#checking for null values
df.isnull().sum()

index     0
tweet    17
label     0
dtype: int64

In [ ]:
#dropping NaN values
df = df.dropna()

In [ ]:
#get the independent features
X = df.drop('label', axis=1)

In [ ]:
#dependent features
y = df['label']

In [ ]:
X.shape

(41851, 2)

In [ ]:
y.shape

(41851,)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
#data preprocessing
ps = PorterStemmer()
def stemming(tweet):
    stemmed_content = re.sub('[^a-zA-Z]',' ',tweet)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

df['tweet'] = df['tweet'].apply(stemming)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
transformer = TfidfVectorizer()
news_transformer = transformer.fit_transform(df["tweet"])

In [ ]:
# Saving model as pickle file.
import pickle
pickle.dump(transformer, open("news-transformer.pkl", "wb"))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
x = df['tweet']
X = transformer.transform(x)
y = df['label']

In [ ]:
#splitting our datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#building our model
model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# accuracy score on dataset
X_test_pred = model.predict(X_test)
test_score = accuracy_score(y_test, X_test_pred)

X_train_pred = model.predict(X_train)
train_score = accuracy_score(y_train, X_train_pred)

# Printing results
print(f"The accuracy on Test data is {round(test_score, 4)}%")
print(f"The accuracy on Training data is {round(train_score, 4)}%")

The accuracy on Test data is 0.9488%
The accuracy on Training data is 0.972%


In [ ]:
#testing the model
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
# Saving model as pickle file
pickle.dump(model, open("pred-model.pkl", "wb"))

In [ ]:
import pickle
model = pickle.load(open("news-transformer.pkl", "rb"))
model_2 = pickle.load(open("pred-model.pkl", "rb"))